# Проверка качества данных: Источник дефолтов

**Таблица:** `sandbox_ai.tmp_defaults_svy`  
**Содержание:** история дефолтов клиентов банка (2009–2025)

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

FILE_PATH = "../sources/data_defaults.pkl"

EXCLUDE_FROM_NULLS = {"reasons_on_last_date_month"}

FIELD_DESCRIPTIONS = {
    "inn": "ИНН клиента. 10 цифр для юридических лиц, 12 цифр для ИП/КФХ",
    "default_reason": "Причина дефолта (код): default_90, def_reserve и т.д.",
    "start_date": "Дата начала дефолта (ДД.ММ.ГГГГ)",
    "cure_date": "Дата выздоровления (выхода из дефолта). Пусто, если клиент ещё в дефолте",
    "finish_date": "Дата окончания (закрытия) дефолта. Пусто, если дефолт не закрыт",
    "sequence_of_defaults": "Последовательность этапов дефолта (номера шагов через ';')",
    "sequence_of_dates": "Даты каждого шага из sequence_of_defaults (через ';')",
    "writeoff": "Признак списания безнадёжной задолженности (0/1)",
    "unlimited_default": "Признак бессрочного (непрекращённого) дефолта (0/1)",
    "reasons_on_last_date_month": "Причина дефолта на последнюю отчётную дату/месяц",
}

In [ ]:
df = pd.read_pickle(FILE_PATH)
df = df.astype(str).replace("nan", np.nan)
df.columns = df.columns.str.strip()

print(f"Загружено строк: {len(df):,}, колонок: {len(df.columns)}")
df.head()

## 1. Список всех полей с описанием

In [ ]:
fields_data = []
for i, col in enumerate(df.columns, start=1):
    desc = FIELD_DESCRIPTIONS.get(col, "(описание отсутствует)")
    fields_data.append({"№": i, "Поле": col, "Описание": desc})

df_fields = pd.DataFrame(fields_data)
df_fields.style.hide(axis="index")

## 2. Пропуски и пустые значения

Колонка `reasons_on_last_date_month` исключена из проверки.

In [ ]:
def is_empty(val):
    if pd.isna(val):
        return True
    s = str(val).strip().lower()
    return s in ("", "nan", "none", "nat")

cols_to_check = [c for c in df.columns if c not in EXCLUDE_FROM_NULLS]
total = len(df)

nulls_data = []
for col in cols_to_check:
    nulls = df[col].apply(is_empty).sum()
    pct = nulls / total * 100
    nulls_data.append({
        "Поле": col,
        "Пропуски": nulls,
        "%": round(pct, 1),
    })

df_nulls = pd.DataFrame(nulls_data)

def highlight_nulls(row):
    if row["Пропуски"] > 0:
        return ["background-color: #fff3cd"] * len(row)
    return [""] * len(row)

df_nulls.style.apply(highlight_nulls, axis=1).hide(axis="index")

## 3. Полные дубликаты

In [ ]:
dup_mask = df.duplicated(keep=False)
n_dup_rows = dup_mask.sum()
n_extra_copies = df.duplicated(keep="first").sum()

print(f"Всего строк в таблице: {len(df):,}")
print(f"Строк, участвующих в дублировании: {n_dup_rows}")
print(f"Из них лишних копий: {n_extra_copies}")

if n_dup_rows > 0:
    dup_df = df[dup_mask].copy()
    dup_df["_group"] = dup_df.groupby(list(df.columns)).ngroup()
    dup_df["_count"] = dup_df.groupby("_group")["_group"].transform("count")
    dup_display = (
        dup_df.drop_duplicates(subset="_group")
        .sort_values("_group")
        .rename(columns={"_count": "Повторений"})
        .drop(columns="_group")
        .reset_index(drop=True)
    )
    print(f"\nДублирующиеся строки:")
    display(dup_display)
else:
    print("\nПолных дубликатов не обнаружено.")